# Obteniendo el **TOTAL de vacunados y fallecidos** por COVID-19 por cada **semana epidemiológica** de todo el Perú

# 0. Run libraries and functions

In [1]:
import pandas as pd
import numpy as np

import functions as fn

In [2]:
def just_dates(vac_url, fal_url):
    """
    Función para filtrar a cada vacunado y fallecido reportado en su año
    y semana epidemiológica (Fallecidos = fal, Vacunados = vac)

    Parameters
    ----------
    vac_url: Directorio o url del dataset de VACUNADOS contra COVID-19
    fal_url: Directorio o url del dataset de FALLECIDOS por COVID-19

    Returns
    -------
    lst_vac: Lista de 'chunks' o dataframes de tamaño n de VACUNADOS
    df_fal: Dataframe de FALLECIDOS

    """
    vac_col = ['fecha_vacunacion','dosis']  # Seleccionar solamente la columna de fechas de ambos datasets                         
    fal_col = ['FECHA_FALLECIMIENTO']
    
    df_vac = fn.read_largeCSV_file(vac_url, ',', vac_col)   # Leemos los datasets       
    df_fal = fn.read_largeCSV_file(fal_url, ';', fal_col)
    
    fn.variable_fecha_ymd(df_vac, 'fecha_vacunacion')   # Transformamos a formato fecha (datetime64[ns])            
    fn.variable_fecha(df_fal, 'FECHA_FALLECIMIENTO')
    fn.date_to_epiweek(df_fal,'FECHA_FALLECIMIENTO')    # Obtenemos semana epidemiológica de fallecidos              
    
    lst_vac = fn.df_into_chunks(df_vac)   #  Dividimos en chunks el dataset de vacunados  

    # Creamos columnas de 1 para contabilizar cada caso de fallecido
    df_fal['fallecido'] = 1                                         
    df_fal['fallecido'].apply(np.int8) 
    df_fal.info()

    # Eliminamos pacientes con 1 dosis (necesitamos 'fully vaccinated people')
    for chunk in lst_vac:                                           
        chunk = chunk.drop(chunk[chunk['dosis'] == 1].index,  inplace=True) 

    # Obtenemos semana epidemiológica de vacunados para cada chunk
    for chunk in lst_vac:                                           
        chunk = fn.date_to_epiweek(chunk, 'fecha_vacunacion')

    # Creamos columnas de 1 en cada chunk para contabilizar cada caso de vacunado
    for chunk in lst_vac:                                        
        chunk['vacunado'] = 1
        chunk['vacunado'].apply(np.int8)
        del chunk['dosis']
        del chunk
    
    return lst_vac, df_fal
    


def epiweeks(df_fal):
    """
    Devuelve un dataframe con el total de FALLECIDOS por semana y año 
    epidemiológico
    """
    epi_fal = pd.crosstab(index=[df_fal['epi_year'],
                                 df_fal['epi_week']],
                          columns=df_fal['fallecido'])
    epi_fal.columns = ['fallecidos']    
    
    return epi_fal
    
  
    
def epiweeks_chunks(dfs_vac):
    """
    Devuelve un dataframe con el total de VACUNADOS por semana y año 
    epidemiológico (recibe una lista de dataframes o chunks)
    """
    var_holder = {}     # Diccionario para guardar nombres                                             
    lst_epi_vac = []    # Lista de dfs para cada sumatoria de chunks
                                         
    for i, chunk in enumerate(dfs_vac):
        var_holder['epi_vac_' + str(i)]= pd.crosstab(index=[chunk['epi_year'],
                                                            chunk['epi_week']],
                                                     columns=chunk['vacunado'])
        lst_epi_vac.append(var_holder['epi_vac_' + str(i)])
        
    # Unimos todos los dfs sumados en uno solo
    merged_epivac = pd.concat(lst_epi_vac, axis=1) 
    epi_vac = pd.DataFrame(merged_epivac.sum(numeric_only=True, axis=1))
    epi_vac.columns = ['vacunados']
    
    return epi_vac
  
    
  
def merged_epiweeks(epi_vac, epi_fal):
    """
    Junta los 2 dataframes resultantes, el del total de vacunados y del total de 
    fallecidos por semana epidemiológica
    """
    # Concatenamos ambos dataframes (hay Nan values)
    merged_epiweeks = pd.concat([epi_fal, epi_vac], axis=1)         
    merged_epiweeks = merged_epiweeks.fillna(value = 0)
    # Cambiamos a Int64 ya que existen Nan values y cambia a float64 automáticamente
    merged_epiweeks = merged_epiweeks.astype('Int64')
    merged_epiweeks = merged_epiweeks.rename({'fallecidos': 'deceased', 'vacunados': 'vaccinated'}, axis=1) 
    
    return merged_epiweeks

## 2. Run code

In [3]:
vac_url = 'RawData/TB_VACUNACION_COVID19.csv'
fal_url = 'RawData/fallecidos_covid.csv'

if __name__ == '__main__':
    dfs_vac,df_fal = just_dates(vac_url, fal_url)
    
    epi_fal = epiweeks(df_fal)
    epi_vac = epiweeks_chunks(dfs_vac)
  
    epiweeks = merged_epiweeks(epi_vac, epi_fal)
    
del vac_url, fal_url

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200246 entries, 0 to 200245
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype
---  ------     --------------   -----
 0   epi_year   200246 non-null  int64
 1   epi_week   200246 non-null  int64
 2   fallecido  200246 non-null  int64
dtypes: int64(3)
memory usage: 4.6 MB


/home/xxotto/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/home/xxotto/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/home/xxotto/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/hom

## 3. Guardar el dataframe

In [4]:
epiweeks.to_csv('Data/TOTAL_vacunados_y_fallecidos_x_semanaEpi.csv')